In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor 


from sklearn.metrics import mean_squared_error

train=pd.read_csv("./data/zhengqi_train.txt",sep='\t')
test=pd.read_csv("./data/zhengqi_test.txt",sep='\t')
train.head()

,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V29,V30,V31,V32,V33,V34,V35,V36,V37,target
0,0.566,0.016,-0.143,0.407,0.452,-0.901,-1.812,-2.360,-0.436,-2.114,...,0.136,0.109,-0.615,0.327,-4.627,-4.789,-5.101,-2.608,-3.508,0.175
1,0.968,0.437,0.066,0.566,0.194,-0.893,-1.566,-2.360,0.332,-2.114,...,-0.128,0.124,0.032,0.600,-0.843,0.160,0.364,-0.335,-0.730,0.676
2,1.013,0.568,0.235,0.370,0.112,-0.797,-1.367,-2.360,0.396,-2.114,...,-0.009,0.361,0.277,-0.116,-0.843,0.160,0.364,0.765,-0.589,0.633
3,0.733,0.368,0.283,0.165,0.599,-0.679,-1.200,-2.086,0.403,-2.114,...,0.015,0.417,0.279,0.603,-0.843,-0.065,0.364,0.333,-0.112,0.206
4,0.684,0.638,0.260,0.209,0.337,-0.454,-1.073,-2.086,0.314,-2.114,...,0.183,1.078,0.328,0.418,-0.843,-0.215,0.364,-0.280,-0.028,0.384


## Gridsearch

In [16]:
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

iris=load_iris()
X_train,X_test,y_train,y_test=train_test_split(iris.data,iris.target,random_state=0)

# grid search start
best_score=0
for gamma in [0.001,0.01,0.1,1,10,100]:
    for C in [0.001,0.01,0.1,1,10,100]:
        svm=SVC(gamma=gamma,C=C)
        svm.fit(X_train,y_train)
        score=svm.score(X_test,y_test)
        if score>best_score:
            best_score=score
            best_parameters={'gamma':gamma,'C':C}
# grid search end
print("best score:{:.2f}".format(best_score))
print("Best parameteres:{}".format(best_parameters))

0.23684210526315788
0.23684210526315788
0.23684210526315788
0.5789473684210527
0.9210526315789473
0.9736842105263158
0.23684210526315788
0.23684210526315788
0.5789473684210527
0.9210526315789473
0.9736842105263158
0.9736842105263158
0.23684210526315788
0.23684210526315788
0.9210526315789473
0.9736842105263158
0.9736842105263158
0.9473684210526315
0.23684210526315788
0.23684210526315788
0.9736842105263158
0.9736842105263158
0.9736842105263158
0.9736842105263158
0.23684210526315788
0.23684210526315788
0.2894736842105263
0.9210526315789473
0.9210526315789473
0.9210526315789473
0.23684210526315788
0.23684210526315788
0.23684210526315788
0.34210526315789475
0.4473684210526316
0.4473684210526316
best score:0.97
Best parameteres:{'gamma': 0.001, 'C': 100}


## GridSearchCV

In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_iris
iris=load_iris()

X_train,X_test,y_train,y_test=train_test_split(iris.data,iris.target,random_state=0)

randomForestRegressor=RandomForestRegressor()
parameters={'n_estimators':[50,100,200],'max_depth':[1,2,3]}
clf=GridSearchCV(randomForestRegressor,parameters,cv=5)
clf.fit(X_train,y_train)
score_test=mean_squared_error(y_test,clf.predict(X_test))
print(score_test)

0.036928024303132946


# 归一化前后预测效果对比

In [3]:
# 归一化前
X_train=np.array(train.drop(['target'], axis = 1))#(2888, 38)
y_train=np.array(train['target'])#(2888,)
X_test=test

In [7]:
# 归一化后
from sklearn.preprocessing import StandardScaler
standardScaler=StandardScaler()

X_train_standard=standardScaler.fit_transform(X_train)
X_test_standard=standardScaler.transform(X_test)

In [11]:
folds=5
kf=KFold(n_splits=folds,shuffle=True,random_state=2019)
for i, (train_index,test_index) in enumerate(kf.split(X_train)):
    lgbr=LGBMRegressor()
    lgbr.fit(X_train[train_index],y_train[train_index])
    y_pred_lgbr=lgbr.predict(X_train[test_index])
    print(i,'th fold：',mean_squared_error(y_pred_lgbr,y_train[test_index]))

0 th fold： 0.11224220616654326
1 th fold： 0.13300272094920537
2 th fold： 0.10605540893432652
3 th fold： 0.1076804629328409
4 th fold： 0.10393625037196488


In [13]:
folds=5
kf=KFold(n_splits=folds,shuffle=True,random_state=2019)
for i, (train_index,test_index) in enumerate(kf.split(X_train_standard)):
    lgbr=LGBMRegressor()
    lgbr.fit(X_train_standard[train_index],y_train[train_index])
    y_pred_lgbr=lgbr.predict(X_train_standard[test_index])
    print(i,'th fold：',mean_squared_error(y_pred_lgbr,y_train[test_index]))

0 th fold： 0.11149853241390828
1 th fold： 0.13314208105691439
2 th fold： 0.10368902671149714
3 th fold： 0.11081503771137614
4 th fold： 0.10163036049650867


In [18]:
from sklearn.linear_model import LinearRegression
LR=LinearRegression()
LR.fit(train_x,train_y)
y_pred_LR=LR.predict(train_x)
print(mean_squared_error(train_y,y_pred_LR))

0.10732587922397853


In [19]:
from sklearn.svm import LinearSVR
LSVR=LinearSVR()
LSVR.fit(train_x,train_y)
y_pred_LSVR=LSVR.predict(train_x)
print(mean_squared_error(train_y,y_pred_LSVR))

0.10922751884017427


C:\Users\nanfe\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [20]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
rf.fit(train_x,train_y)
y_pred_rf=rf.predict(train_x)
print(mean_squared_error(train_y,y_pred_rf))

0.01692468995141967


In [22]:
from sklearn.ensemble import GradientBoostingRegressor
gbrt=GradientBoostingRegressor()
gbrt.fit(train_x,train_y)
y_pred_gbrt=gbrt.predict(train_x)
print(mean_squared_error(train_y,y_pred_gbrt))

0.07352460526625344


In [5]:
from xgboost import XGBRegressor
xgbr=XGBRegressor()
xgbr.fit(train_x,train_y)
y_pred_xgbr=xgbr.predict(train_x)
print(mean_squared_error(train_y,y_pred_xgbr))

0.0015769715398205853


In [4]:
from lightgbm import LGBMRegressor 
lgbr=LGBMRegressor()
lgbr.fit(train_x,train_y)
y_pred_lgbr=lgbr.predict(train_x)
print(mean_squared_error(train_y,y_pred_lgbr))

0.01988448016554369


In [6]:
from catboost import CatBoostRegressor
catbr=CatBoostRegressor()
catbr.fit(train_x,train_y)
y_pred_catbr=catbr.predict(train_x)
print(mean_squared_error(train_y,y_pred_catbr))

Learning rate set to 0.048412
0:	learn: 0.9506008	total: 152ms	remaining: 2m 31s
1:	learn: 0.9175590	total: 154ms	remaining: 1m 17s
2:	learn: 0.8878563	total: 157ms	remaining: 52s
3:	learn: 0.8577366	total: 159ms	remaining: 39.5s
4:	learn: 0.8289625	total: 161ms	remaining: 32s
5:	learn: 0.8013219	total: 163ms	remaining: 27s
6:	learn: 0.7751888	total: 165ms	remaining: 23.5s
7:	learn: 0.7518653	total: 167ms	remaining: 20.8s
8:	learn: 0.7295730	total: 170ms	remaining: 18.7s
9:	learn: 0.7080797	total: 172ms	remaining: 17s
10:	learn: 0.6868655	total: 174ms	remaining: 15.6s
11:	learn: 0.6672122	total: 176ms	remaining: 14.5s
12:	learn: 0.6485303	total: 178ms	remaining: 13.5s
13:	learn: 0.6314525	total: 180ms	remaining: 12.7s
14:	learn: 0.6160128	total: 183ms	remaining: 12s
15:	learn: 0.6019887	total: 185ms	remaining: 11.4s
16:	learn: 0.5869445	total: 188ms	remaining: 10.8s
17:	learn: 0.5742662	total: 190ms	remaining: 10.4s
18:	learn: 0.5614595	total: 192ms	remaining: 9.92s
19:	learn: 0.548943

182:	learn: 0.2849946	total: 541ms	remaining: 2.42s
183:	learn: 0.2846655	total: 543ms	remaining: 2.41s
184:	learn: 0.2843059	total: 546ms	remaining: 2.4s
185:	learn: 0.2840162	total: 548ms	remaining: 2.4s
186:	learn: 0.2836803	total: 550ms	remaining: 2.39s
187:	learn: 0.2833659	total: 552ms	remaining: 2.38s
188:	learn: 0.2830575	total: 555ms	remaining: 2.38s
189:	learn: 0.2827824	total: 557ms	remaining: 2.37s
190:	learn: 0.2824489	total: 559ms	remaining: 2.37s
191:	learn: 0.2820090	total: 561ms	remaining: 2.36s
192:	learn: 0.2817084	total: 563ms	remaining: 2.35s
193:	learn: 0.2812486	total: 566ms	remaining: 2.35s
194:	learn: 0.2807423	total: 568ms	remaining: 2.34s
195:	learn: 0.2804464	total: 570ms	remaining: 2.34s
196:	learn: 0.2801435	total: 572ms	remaining: 2.33s
197:	learn: 0.2797956	total: 574ms	remaining: 2.32s
198:	learn: 0.2793034	total: 576ms	remaining: 2.32s
199:	learn: 0.2790279	total: 578ms	remaining: 2.31s
200:	learn: 0.2785893	total: 580ms	remaining: 2.31s
201:	learn: 0.

353:	learn: 0.2306978	total: 901ms	remaining: 1.64s
354:	learn: 0.2302979	total: 903ms	remaining: 1.64s
355:	learn: 0.2300959	total: 905ms	remaining: 1.64s
356:	learn: 0.2297835	total: 907ms	remaining: 1.63s
357:	learn: 0.2296129	total: 909ms	remaining: 1.63s
358:	learn: 0.2292437	total: 912ms	remaining: 1.63s
359:	learn: 0.2289944	total: 914ms	remaining: 1.62s
360:	learn: 0.2286586	total: 916ms	remaining: 1.62s
361:	learn: 0.2283668	total: 918ms	remaining: 1.62s
362:	learn: 0.2281173	total: 920ms	remaining: 1.61s
363:	learn: 0.2278242	total: 922ms	remaining: 1.61s
364:	learn: 0.2275632	total: 924ms	remaining: 1.61s
365:	learn: 0.2273325	total: 926ms	remaining: 1.6s
366:	learn: 0.2269228	total: 929ms	remaining: 1.6s
367:	learn: 0.2265822	total: 931ms	remaining: 1.6s
368:	learn: 0.2263414	total: 933ms	remaining: 1.59s
369:	learn: 0.2260154	total: 935ms	remaining: 1.59s
370:	learn: 0.2257702	total: 937ms	remaining: 1.59s
371:	learn: 0.2255814	total: 939ms	remaining: 1.58s
372:	learn: 0.2

526:	learn: 0.1928189	total: 1.26s	remaining: 1.13s
527:	learn: 0.1927204	total: 1.26s	remaining: 1.13s
528:	learn: 0.1925392	total: 1.26s	remaining: 1.13s
529:	learn: 0.1923698	total: 1.27s	remaining: 1.12s
530:	learn: 0.1921147	total: 1.27s	remaining: 1.12s
531:	learn: 0.1919553	total: 1.27s	remaining: 1.12s
532:	learn: 0.1918305	total: 1.27s	remaining: 1.12s
533:	learn: 0.1916195	total: 1.28s	remaining: 1.11s
534:	learn: 0.1913590	total: 1.28s	remaining: 1.11s
535:	learn: 0.1910674	total: 1.28s	remaining: 1.11s
536:	learn: 0.1908961	total: 1.28s	remaining: 1.1s
537:	learn: 0.1907328	total: 1.28s	remaining: 1.1s
538:	learn: 0.1906609	total: 1.29s	remaining: 1.1s
539:	learn: 0.1904848	total: 1.29s	remaining: 1.1s
540:	learn: 0.1902083	total: 1.29s	remaining: 1.09s
541:	learn: 0.1900810	total: 1.29s	remaining: 1.09s
542:	learn: 0.1898414	total: 1.29s	remaining: 1.09s
543:	learn: 0.1895310	total: 1.3s	remaining: 1.09s
544:	learn: 0.1894145	total: 1.3s	remaining: 1.08s
545:	learn: 0.1891

688:	learn: 0.1642802	total: 1.62s	remaining: 732ms
689:	learn: 0.1641074	total: 1.62s	remaining: 730ms
690:	learn: 0.1640042	total: 1.63s	remaining: 727ms
691:	learn: 0.1637909	total: 1.63s	remaining: 725ms
692:	learn: 0.1637034	total: 1.63s	remaining: 722ms
693:	learn: 0.1635235	total: 1.63s	remaining: 720ms
694:	learn: 0.1632682	total: 1.63s	remaining: 717ms
695:	learn: 0.1630523	total: 1.64s	remaining: 715ms
696:	learn: 0.1628728	total: 1.64s	remaining: 713ms
697:	learn: 0.1626466	total: 1.64s	remaining: 710ms
698:	learn: 0.1625377	total: 1.64s	remaining: 708ms
699:	learn: 0.1623601	total: 1.65s	remaining: 705ms
700:	learn: 0.1621674	total: 1.65s	remaining: 703ms
701:	learn: 0.1620811	total: 1.65s	remaining: 701ms
702:	learn: 0.1618598	total: 1.65s	remaining: 698ms
703:	learn: 0.1616215	total: 1.65s	remaining: 696ms
704:	learn: 0.1615603	total: 1.66s	remaining: 693ms
705:	learn: 0.1614135	total: 1.66s	remaining: 691ms
706:	learn: 0.1612122	total: 1.66s	remaining: 688ms
707:	learn: 

858:	learn: 0.1398754	total: 1.98s	remaining: 325ms
859:	learn: 0.1397657	total: 1.98s	remaining: 323ms
860:	learn: 0.1396024	total: 1.99s	remaining: 321ms
861:	learn: 0.1394880	total: 1.99s	remaining: 318ms
862:	learn: 0.1393271	total: 1.99s	remaining: 316ms
863:	learn: 0.1392288	total: 1.99s	remaining: 314ms
864:	learn: 0.1391526	total: 1.99s	remaining: 311ms
865:	learn: 0.1389682	total: 2s	remaining: 309ms
866:	learn: 0.1388151	total: 2s	remaining: 307ms
867:	learn: 0.1387412	total: 2s	remaining: 304ms
868:	learn: 0.1386488	total: 2s	remaining: 302ms
869:	learn: 0.1385583	total: 2s	remaining: 300ms
870:	learn: 0.1384079	total: 2.01s	remaining: 297ms
871:	learn: 0.1382794	total: 2.01s	remaining: 295ms
872:	learn: 0.1381938	total: 2.01s	remaining: 293ms
873:	learn: 0.1380640	total: 2.01s	remaining: 290ms
874:	learn: 0.1379892	total: 2.02s	remaining: 288ms
875:	learn: 0.1378314	total: 2.02s	remaining: 286ms
876:	learn: 0.1376772	total: 2.02s	remaining: 283ms
877:	learn: 0.1375089	total

In [7]:
result=catbr.predict(test)
pd.DataFrame(result).to_csv('result_20220419.txt',index=False,header=False)# 得分：0.1381